# ゼロから作る Deep Learning

## 概要

これは書籍で学んだことをメモするためのノートです。

## 始めに

必要なライブラリをインポートします

- numpy (行列計算)
- matplotlib.pyplot (グラフ描画)
- PIL.Image (画像表示)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image

## 準備運動

- numpyを使用してグラフの描画
- matplotlib.pyplotを使用して画像の描画

In [ ]:
x = np.arange(0, 6, 0.1)

plt.plot(x, np.sin(x), label='sin')
plt.plot(x, np.cos(x), label='cos')
plt.xlabel('x')
plt.xlabel('y')
plt.title('sin & cos')
plt.legend()
plt.show()

plt.imshow(Image.open('img/penguins.jpeg'))

## パーセプトロン

入力値に応じて0か1を返す関数で以下の定義で表される

- `n` : 入力の数

$$
y = \begin{cases}
  0 & x \cdot w + b \lt 0 \\
  1 & x \cdot w + b \ge 0
\end{cases}
$$

出力値をグラフで表すと以下のような線になる

In [ ]:
def step (x) :
    return 0 if x < 0 else 1

def total(x, w, b) :
    return np.dot(x, w) + b

def perseptron(x, w, b) :
    return step(total(x, w, b))

x = np.arange(0, 2, 0.001).reshape(200, 10)
w = np.random.rand(10)
b = -4
plt.ylim(ymin=-0.2, ymax=1.2)
plt.plot([perseptron(x, w, b) for x in x])
plt.show()

## ニューラルネットワーク

パーセプトロンの定義を以下のように書き換える

$$
y = h(x \cdot w + b)
$$

$$
h(a) = \begin{cases}
  0 & a \lt 0 \\
  1 & a \ge 0
\end{cases}
$$

`h(x)`のことを__活性化関数__と呼ぶ

ニューラルネットワークでは活性化関数に以下の関数を使用する

- シグモイド関数
- ReLU

※ パーセプトロンで使用した関数は__ステップ関数__という

### シグモイド関数

$$
h(x) = \frac{1}{1 + \mathrm{e}^{-x}}
$$

### ReLU

$$
h(a) = \begin{cases}
  0 & a \lt 0 \\
  a & a \ge 0
\end{cases}
$$

シグモイド関数、ReLU、ステップ関数を比較すると以下のような線になる

In [ ]:
def sigmoid(x) :
    return 1 / (1 + np.exp(-x))

def relu(x) :
    return np.maximum(0, x)

def neuralnet(x, w, b, func) :
    return func(total(x, w, b))

plt.ylim(ymin=-0.2, ymax=1.2)
plt.plot([neuralnet(x, w, b, sigmoid) for x in x], label='sigmoid')
plt.plot([neuralnet(x, w, b, relu) for x in x], label='ReLU')
plt.plot([perseptron(x, w, b) for x in x], label='perseptron', linestyle='--')
plt.legend()
plt.show()

## 多層ニューラルネットワーク

これまでは[入力]と[出力]だけの2層のネットワークを見てきたが、多層のネットワークも構築可能である

層が増えれば複雑さが増すためより高度な分類が行える

以下に4層ニューラルネットワークの例を示す

In [ ]:
x = np.arange(0, 2, 0.0001).reshape(200, 10, 10)
w = [np.random.rand(10), np.random.rand(10), np.random.rand(1)]
b = [-4, -7, -7]

plt.plot([neuralnet(neuralnet(neuralnet(x, w[0], b[0], sigmoid), w[1], b[1], sigmoid), w[2], b[2], sigmoid) for x in x])
plt.show()

## 出力関数

最後の層で使用する活性関数のことを特別に出力関数という

出力関数には以下の関数がある

- 恒等関数
- シグモイド関数
- ソフトマックス関数

### 恒等関数

常に出力の値が入力値と等しくなる関数

$$
y_k = a_k
$$

### ソフトマックス関数

全ての出力層の合計値が1になるため、各ノードの出力値を確立として扱うことができる

$$
y_k = \frac{\mathrm{e}^{a_k}}{\sum^n_{i=1} \mathrm{e}^{a_i}}
$$

In [ ]:
def softmax(x) :
    exp = np.exp(x - np.max(x))
    return exp / np.sum(exp)

x = np.arange(0, 2, 0.0001).reshape(200, 10, 10)
w = [np.random.rand(10), np.random.rand(10, 4), np.random.rand(4, 5)]
b = [-4, -7, -1]

plt.plot([neuralnet(neuralnet(neuralnet(x, w[0], b[0], sigmoid), w[1], b[1], sigmoid), w[2], b[2], softmax) for x in x])
plt.title('Softmax')
plt.show()

## 損失関数

出力層で出力された値と実際の値の差異を表し、損失関数で得られたからどのように学習させるかを決めるための指標を得る

具体的には以下のグラフが示すように損失関数の出力は曲線を描くため、微分を用いて傾きを求めることでどの方向に値を調整するかを把握することができる

- 2乗和誤差
- 交差エントロピー誤差

ここでは実際の値(教師データ)を`t`とする

### 2乗和誤差

$$
E = \frac{1}{2} \sum_k (y_k - t_k)^2
$$

### 交差エントロピー誤差

$$
E = - \sum_k t_k \log y_k
$$

In [ ]:
def conv_2d(array) :
    array[:] = np.array(array)
    return array if array.ndim > 1 else np.reshape(array, (1, array.size))

def mean_squared_error(y, t) :
    return np.sum((conv_2d(y) - np.array(t)) ** 2, axis=1) / 2

def cross_entropy_error(y, t) :
    return -np.sum(np.array(t) * np.log(conv_2d(y) + 1e-7), axis=1)

y = np.array([
    np.zeros(200),
    np.zeros(200),
    np.zeros(200),
    np.arange(1, 0, -0.005),
    np.arange(0, 1, 0.005),
]).T
t = [0, 0 ,0 , 0, 1]

plt.plot(mean_squared_error(y, t), label='mean squared')
plt.legend()
plt.show()
plt.plot(cross_entropy_error(y, t), label='cross entropy')
plt.legend()
plt.show()

## 微分

微分は傾きが変化する関数の中で特定の時点での傾きを求める

$$
\frac{df(x)}{dx} = \lim_{h \to 0} \frac{f(x + h) - f(x)}{h}
$$

## 勾配

1つの変数以外を定数と仮定して、その変数に対して微分することを__偏微分__といい、偏微分をすべての変数に対して行った結果をベクトルとしてまとめたものを勾配という

$$
\left(
  \frac{\partial f(x_0)}{\partial x_0},
  \frac{\partial f(x_1)}{\partial x_1},
  \cdots
  \frac{\partial f(x_n)}{\partial x_n}
\right)
$$

In [ ]:
def numerical_diff(f, x) :
    h = 1e-4
    return (f(x + h) - f(x - h)) / (2 * h)

def numerical_gradient(f, x) :
    h = 1e-4
    x = conv_2d(x)
    grad = np.zeros_like(x)
    for i in range(x.shape[0]) :
        for j in range(x.shape[1]) :
            tmp = x[i][j]
            x[i][j] = tmp + h
            a = f(x[i])

            x[i][j] = tmp - h
            b = f(x[i])
            grad[i][j] = (a - b) / (2 * h)

            x[i][j] = tmp
    return grad

def test_func1(x) :
    return 0.01 * x ** 2 + 0.1 * x

def test_func2(x) :
    return np.sum(conv_2d(x) ** 2, axis=1)

x = np.arange(-50., 50., 1.)
plt.plot(x, test_func1(x), label='y = 0.01x^2 + 0.1x')
plt.plot(x, numerical_diff(test_func1, x), label='numerical diff', linestyle='--')
plt.legend()
plt.show()

x = np.meshgrid(np.arange(-10., 10., 1.), np.arange(-10., 10., 1.))
x = [x[0].flatten(), x[1].flatten()]
y = numerical_gradient(test_func2, np.array(x))
plt.quiver(x[0], x[1], -y[0], -y[1], angles="xy",color="#666666")
plt.title('gradient')
plt.show()

### 学習率

勾配を用いることである地点の傾きを得ることができる。

傾きはパラメータを変化させたときにどのように出力値が変わるかを示すため、値を減らす方向にパラメータを動かすことで誤差を減らすことができる

パラメータの変化量のことを__学習率(η)__という

$$
x_k = x_k - \eta \frac{\partial f(x_k)}{\partial x_k}
$$

In [ ]:
def gradient_descent(f, x, lr=0.01, step=100) :
    history = []
    x = conv_2d(x)

    for i in range(step) :
        grad = numerical_gradient(f, x)
        x -= lr * grad
        history.append(x.copy())

    return x, np.array(history).T

x, history = gradient_descent(test_func2, np.array([-3.0, 4.0]), lr=0.1)

plt.plot(history[0], history[1], 'o')
plt.ylim(ymin=-5, ymax=5)
plt.xlim(xmin=-5, xmax=5)
plt.show()

## ニューラルネットワークの実装

ニューラルネットワークで学習をするためにどのように処理を進めていくかをいかに示します

1. 重み(w)の初期値と入力値を決める
2. ニューラルネットワークの各レイヤーに対して活性関数を適用する
  - 出力層 : ソフトマックス関数
  - 出力層以外 : シグモイド関数
3. 出力値に対して損失関数を適用する
4. [2] - [3]をすべてのパラメータに対して適用し勾配を求める
5. パラメータを更新して[2] - [4]を繰り返す

In [ ]:
def init_network(shape) :
    i = 0
    w = []
    b = []

    for curr in shape :
        if i > 0 :
            w.append(np.random.rand(prev, curr))
            b.append(np.random.rand(curr))

        prev = curr
        i += 1

    return w, b

def gradient_network(x, w, b, t, step=100, lr=0.01) :
    history = []
    loss_func = lambda arg : loss(x, w, b, t)

    for i in range(step) :
        grad = []

        for (_w, _b) in zip(w, b) :
            grad.append((numerical_gradient(loss_func, _w), numerical_gradient(loss_func, _b)))

        for (_w, _b, _grad) in zip(w, b, grad) :
            _w -= lr * _grad[0]
            _b -= lr * _grad[1][0]
        history.append(loss(x, w, b, t))

    return history

def loss(x, w, b, t) :
    for (_w, _b) in zip(w[:-1], b[:-1]) :
        x = neuralnet(x, _w, _b, sigmoid)

    y = neuralnet(x, w[-1], b[-1], softmax)
    return cross_entropy_error(y, t)

x = np.random.rand(4)
w, b = init_network((4, 2, 3, 5))
t = [0, 0, 0, 1, 0]
history = gradient_network(x, w, b, t, step=500)

plt.plot(history, label='cross entropy error')
plt.legend()
plt.show()

## 誤差逆伝搬法

微分を単純な計算の集合として考えることで、計算を簡略化する

$$
y = ab + c
$$

このような関数があった場合、この関数を以下のように分解する

$$
y = A + c \\
A = ab
$$

この関数内のすべての微分は以下のようになる

$$
\frac{dA}{da} = b \\
\frac{dA}{db} = a \\
\frac{dy}{dA} = 1 \\
\frac{dy}{dc} = 1
$$

それぞれの変数に対する微分を示す

- aに対する微分
$$
\frac{dy}{da} = \frac{dA}{da} \frac{dy}{dA} = b \times 1 = b
$$

- bに対する微分
$$
\frac{dy}{db} = \frac{dA}{db} \frac{dy}{dA} = a \times 1 = a
$$

- cに対する微分
$$
\frac{dy}{dc} = 1
$$

このように微分を計算する際は逆順に算出するため__逆伝搬__という

### 加算の場合

$$
y = a + b
$$

上の計算を考える場合、a、b両方に対する微分の結果は1になる

### 乗算の場合

$$
y = ab
$$

上の計算を考える場合、aに対する微分はbに、bに対する微分はaになることから入力値を反転させた値が微分の結果になる


## 活性関数の逆伝搬

### ReLU

$$
y = \begin{cases}
  0 & x \lt 0 \\
  x & x \ge 0
\end{cases}
$$

上記の定義からReLUの微分は

$$
\frac{dy}{dx} = \begin{cases}
  0 & x \lt 0 \\
  1 & x \ge 0
\end{cases}
$$

となるため、xが負数となる場合は以降の逆伝搬の算出は不要となり、計算量を軽減することができる

### シグモイド関数

$$
y = \frac{1}{1 + \mathrm{e}^{-x}}
$$

を分解すると

$$
y = \frac{1}{A} \\
A = 1 + B \\
B = \mathrm{e}^{C} \\
C = -x
$$

となり、それぞれの微分は

$$
\frac{dC}{dx} = -1 \\
\frac{dB}{dC} = \mathrm{e}^{C} \\
\frac{dA}{dB} = 1 \\
\frac{dy}{dA} = -\frac{1}{A^2} = -y^2
$$

である。これを計算すると

$$
\frac{dy}{dx} = y^2\mathrm{e}^{-x}
$$

となり微分の結果、入力値`x`と出力値`y`から算出可能であるため途中の計算を省略できる

更に、`y`は`x`から導出されるため`y`のみから算出できる。

$$
y^2\mathrm{e}^{-x} = y(1 - y)
$$

__※ シグモイド関数で`x`の低を`e`にしていた理由は`e`が微分しても`e`になるという性質を持っていたため計算が容易になるという理由だと思われる__

In [ ]:
class ReLU :
    def __init__(self) :
        self.out = None

    def forward(self, x) :
        self.out = relu(x)
        return self.out

    def backword(self, dout) :
        return (self.out > 0) * dout

class Sigmoid :
    def __init__(self) :
        self.out = None
        
    def forward(self, x) :
        self.out = sigmoid(x)
        return self.out

    def backword(self, dout) :
        return dout * self.out * (1. - self.out)

## 入力値の逆伝搬

ニューラルネットワークのパラメータは

$$
y = x \cdot w + b
$$

で計算されるため、これを分解すると

$$
y = A + b \\
A = x \cdot w
$$

となり、それぞれの微分は

$$
\frac{dA}{dx} = w^T \\
\frac{dy}{dA} = 1
$$

となり、これを計算すると

$$
\frac{dy}{dx} = w^T
$$

で表すことができる

In [27]:
class Params:
    def __init__(self) :
        self.w = None

    def forward(self, x, w, b) :
        self.w = w
        return total(x, w, b)

    def backword(self, dout) :
        return np.dot(dout, self.w.T)